In [3]:
import pandas as pd
import openml
import glob
from pymfe.mfe import MFE

c:\Users\wiche\OneDrive\Bureaublad\MasterThesis\masterthesisenv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from itertools import compress

datasets = openml.datasets

In [4]:
from metafeature_mapping import mapping_general, mapping_infotheory, mapping_stat

In [5]:
mapping = mapping_general | mapping_stat | mapping_infotheory

ftrs = list(mapping.keys())

In [18]:
mfe_landmark = MFE(groups = ["landmarking"], summary = ["nanmean", "nansd"], num_cv_folds = 5, lm_sample_frac = 0.5, measure_time = "total_summ", suppress_warnings=True)
mfe = MFE(features = ftrs, summary = ["nanmean", "nansd"], measure_time = "total_summ", suppress_warnings=True)
columns = list(mfe.extract_metafeature_names())
columns_lm = list(mfe.extract_metafeature_names())
columns.extend(columns_lm)

In [20]:
dataset = datasets.get_dataset(2, download_all_files=False)
X, y, cat_mask, _ = dataset.get_data(dataset_format="dataframe", target = dataset.default_target_attribute)
arr_X = X.to_numpy()
arr_y = y.to_numpy()
num_mask = [not elem for elem in cat_mask]
cat_cols = list(compress(list(X.columns), cat_mask))
num_cols = list(compress(list(X.columns), num_mask))
N = X.loc[:, num_cols]
C = X.loc[:, cat_cols]
C = C.apply(lambda x: x.fillna(x.mode(), axis = 0))
N = N.apply(lambda x: x.fillna(x.mean(), axis = 0))

In [21]:
imputed_X = C.merge(N, left_index = True, right_index = True).to_numpy()
mfe_landmark.fit(imputed_X, arr_y)
mfe.fit(arr_X, arr_y)
_, vals, time = mfe.extract()
_, vals_lm, time = mfe_landmark.extract()
vals.extend(vals_lm)

c:\Users\wiche\OneDrive\Bureaublad\MasterThesis\masterthesisenv\lib\site-packages\pymfe\_internal.py:1568: UserWarning: It is not possible make equal discretization
  warnings.warn("It is not possible make equal discretization")


ValueError: Categorical data encoding of type 'gray' has no support for missing values. Please handle the missing data manually before fitting it into the MFE model.

In [65]:
from sklearn.impute import SimpleImputer
import numpy as np

In [102]:
mfe_landmark = MFE(groups = ["landmarking"], summary = ["nanmean", "nansd"], num_cv_folds = 5, lm_sample_frac = 0.5, measure_time = "total_summ", suppress_warnings=True)
mfe = MFE(features = ftrs, summary = ["nanmean", "nansd"], measure_time = "total_summ", suppress_warnings=True)
columns = list(mfe.extract_metafeature_names())
columns_lm = list(mfe.extract_metafeature_names())
columns.extend(columns_lm)


dataset = datasets.get_dataset(2)
X, y, cat_mask, _ = dataset.get_data(dataset_format="array", target = dataset.default_target_attribute)
num_mask = [not elem for elem in cat_mask]
C = X[:, cat_mask]
N = X[:, num_mask]
imputer_cat = SimpleImputer(strategy='most_frequent', missing_values=np.nan)
if len(C) > 0:
    C = imputer_cat.fit_transform(C)

imputer_num = SimpleImputer(strategy='mean', missing_values=np.nan)
N = imputer_num.fit_transform(N)
imputed_X = np.concatenate((C, N), axis = 1)
mfe_landmark.fit(imputed_X, y)
mfe.fit(X, y)
_, vals, time = mfe.extract()
_, vals_lm, time = mfe_landmark.extract()
vals.extend(vals_lm)



c:\Users\wiche\OneDrive\Bureaublad\MasterThesis\masterthesisenv\lib\site-packages\pymfe\_internal.py:1568: UserWarning: It is not possible make equal discretization
  warnings.warn("It is not possible make equal discretization")
c:\Users\wiche\OneDrive\Bureaublad\MasterThesis\masterthesisenv\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\wiche\OneDrive\Bureaublad\MasterThesis\masterthesisenv\lib\site-packages\pymfe\_internal.py:1568: UserWarning: It is not possible make equal discretization
  warnings.warn("It is not possible make equal discretization")
c:\Users\wiche\OneDrive\Bureaublad\MasterThesis\masterthesisenv\lib\site-packages\sklearn\preprocessing\_encoders.py:808: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.war

In [78]:
dataset = datasets.get_dataset(45069, download_all_files=False)
X, y, cat_mask, _ = dataset.get_data(dataset_format="dataframe", target = dataset.default_target_attribute)

In [79]:
X

,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,...,examide,citoglipton,insulin,glyburide.metformin,glipizide.metformin,glimepiride.pioglitazone,metformin.rosiglitazone,metformin.pioglitazone,change,diabetesMed
0,1,3,7,4,41,2,17,0,0,0,...,No,No,Steady,No,No,No,No,No,Ch,Yes
1,5,1,1,3,64,0,10,0,0,1,...,No,No,Steady,No,No,No,No,No,Ch,Yes
2,2,1,1,2,39,0,8,0,0,0,...,No,No,Steady,No,No,No,No,No,Ch,Yes
3,3,1,1,1,39,2,12,0,0,1,...,No,No,No,No,No,No,No,No,No,No
4,3,1,1,3,21,2,23,1,0,0,...,No,No,No,No,No,No,No,No,Ch,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101761,1,1,7,2,52,0,12,0,0,0,...,No,No,No,No,No,No,No,No,No,No
101762,1,1,7,6,50,0,17,1,0,6,...,No,No,Steady,No,No,No,No,No,No,Yes
101763,3,1,1,2,42,3,12,0,0,0,...,No,No,Steady,No,No,No,No,No,No,Yes
101764,1,22,2,14,65,2,18,0,0,0,...,No,No,Down,No,No,No,No,No,Ch,Yes


In [66]:
import pandas as pd
import glob
csv_files = glob.glob(f"../data/final/*.{'csv'}")
meta_features = pd.concat([pd.read_csv(f, index_col = 0) for f in csv_files])

In [71]:
failed_jobs = meta_features[meta_features.isnull().all(1)].index

In [72]:
succeeded = []
for ind in failed_jobs:
    try:
        dataset = datasets.get_dataset(ind)
        X, y, cat_mask, _ = dataset.get_data(dataset_format="dataframe", target = dataset.default_target_attribute)
        succeeded.append(ind)
    except:
        print("failed", ind)

failed 1092
failed 4537


failed 40590
failed 40864


failed 41022
failed 41707


failed 41708
failed 41709


failed 41710


...

failed 41906


failed 42334


failed 42855


failed 43147


failed 43148
failed 43149


In [80]:
additional_df_runs = pd.read_csv("../data/final/additional_parses.csv", index_col = 0)

In [99]:
meta_features = meta_features.dropna(how = 'all')
meta_features.append(additional_df_runs.dropna(how = 'all').drop(4102))

C:\Users\wiche\AppData\Local\Temp\ipykernel_19116\2142291790.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  meta_features.append(additional_df_runs.dropna(how = 'all').drop(4102))


,attr_conc.nanmean,attr_conc.nansd,attr_ent.nanmean,attr_ent.nansd,class_conc.nanmean,class_conc.nansd,cor.nanmean,cor.nansd,cov.nanmean,cov.nansd,...,best_node.mean,best_node.sd,linear_discr.mean,linear_discr.sd,naive_bayes.mean,naive_bayes.sd,random_node.mean,random_node.sd,worst_node.mean,worst_node.sd
2,0.297761,0.242172,0.278180,0.720669,0.408932,0.181142,0.138039,0.129664,3549.620236,17524.065606,...,0.788414,0.001061,0.754981,0.048614,0.412010,0.064147,0.788414,0.001061,0.750562,0.005696
3,0.221499,0.243155,0.510477,0.412594,0.200833,0.232142,0.080608,0.097770,0.010702,0.018146,...,0.660837,0.013755,0.933650,0.019657,0.611407,0.012878,0.660837,0.013755,0.507504,0.011457
4,0.500000,0.000000,0.000000,0.000000,0.500000,0.000000,NaN,NaN,NaN,NaN,...,0.786667,0.229250,0.753333,0.200832,0.653333,0.313227,0.533333,0.102740,0.573333,0.246531
5,0.135112,0.212751,1.340114,1.266438,0.117110,0.152737,0.089051,0.120026,6.686123,28.727778,...,0.579614,0.016241,0.420193,0.044915,0.150435,0.032881,0.531014,0.005185,0.531014,0.005185
6,0.029150,0.025628,2.974428,0.255379,0.099577,0.074523,0.182896,0.208834,0.949368,1.150810,...,0.075100,0.001517,0.698300,0.011972,0.633200,0.009648,0.068300,0.001304,0.054300,0.002775
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45059,0.006091,0.011828,1.122060,1.526477,0.004582,0.010276,0.063266,0.105654,4.758392,55.057110,...,0.932947,0.001421,0.927796,0.006572,0.310144,0.031627,0.932947,0.001421,0.932947,0.001421
45063,0.040851,0.123876,1.823007,1.046028,0.069594,0.142304,0.073516,0.116590,106.788948,2451.274090,...,0.864895,0.030104,0.864895,0.033807,0.806480,0.075525,0.542937,0.006809,0.539860,0.011319
45067,0.013301,0.044547,2.353503,1.589759,0.009729,0.016299,0.002875,0.010615,0.000217,0.009065,...,0.723347,0.000057,0.727020,0.004889,0.207676,0.041234,0.723347,0.000057,0.723272,0.000207
45077,0.044255,0.068679,1.979658,1.203136,0.023764,0.018124,0.083812,0.130393,0.147072,1.026681,...,0.741887,0.017849,0.838688,0.040478,0.628068,0.019782,0.667942,0.016117,0.652758,0.004226


In [90]:
from pymfe.mfe import MFE
mfe = MFE(features = ["random_node", "best_node", "worst_node", "linear_discr", "naive_bayes"], groups = ["landmarking"])
import openml
datasets = openml.datasets

In [92]:
dataset = datasets.get_dataset(4102)
X, y, cat_mask, _ = dataset.get_data(dataset_format="dataframe", target = dataset.default_target_attribute)
X, y = X.to_numpy(), y.to_numpy()

In [1]:
import pandas as pd
mf = pd.read_csv("meta_features_total.csv", index_col = 0)

In [2]:
numpy_mf = mf.values

In [3]:
from meta_func import BackFiller
filler = BackFiller()

In [11]:
mf.isnull().sum(1).sort_values()[-50:]

C:\Users\wiche\AppData\Local\Temp\ipykernel_19116\3890897572.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  mf.isnull().sum(1).sort_values()[-50:]


814       3
811       3
968       3
801       3
741       3
905       3
867       3
1525      3
1460      3
790       3
1013      3
860       3
457       3
865       3
791       3
895       3
530      11
52       11
42554    11
8        11
40916    11
232      11
210      11
27       14
899      14
186      14
957      14
1023     14
172      14
42585    14
42       14
460      23
1008     23
56       23
4        23
42347    25
42399    25
42492    25
42546    25
42547    25
42575    25
42555    25
42558    25
42689    25
1456     25
42694    25
42702    25
42703    25
42548    25
42401    25
dtype: int64

In [63]:
landmarkers = [item for item in list(mfe.extract_metafeature_names()) if item not in  ["elite_nn.mean", "elite_nn.sd", "one_nn.mean", "one_nn.sd"]]

In [56]:
dt = pd.read_csv("../data/times_final/meta_features_classification_2.csv", index_col = 0)

FileNotFoundError: [Errno 2] No such file or directory: '../data/times_final/meta_features_classification_2.csv'

In [60]:
dt.mean(axis = 0).sort_values()[50:]

freq_class.nansd         0.001395
one_nn.nanmean           0.001546
class_conc.nansd         0.002574
kurtosis.nansd           0.003593
one_nn.nanmean.1         0.004987
random_node.nansd        0.005433
nr_bin                   0.008101
best_node.nansd.1        0.008128
nr_outliers              0.009425
worst_node.nansd         0.013163
naive_bayes.nansd        0.013564
linear_discr.nansd.1     0.023363
attr_conc.nansd          0.042089
linear_discr.nansd       0.081403
attr_conc.nanmean        0.130867
elite_nn.nansd.1         0.691515
elite_nn.nansd           3.357778
one_nn.nansd.1           9.425191
one_nn.nansd            49.946412
Unnamed: 0              78.750000
dtype: float64

In [36]:
features = ["best_node", "linear_discr", "naive_bayes", "random_node", "worst_node"]
mfe = MFE(groups = ["landmarking"])
mfe.extract_metafeature_names()

('best_node.mean',
 'best_node.sd',
 'elite_nn.mean',
 'elite_nn.sd',
 'linear_discr.mean',
 'linear_discr.sd',
 'naive_bayes.mean',
 'naive_bayes.sd',
 'one_nn.mean',
 'one_nn.sd',
 'random_node.mean',
 'random_node.sd',
 'worst_node.mean',
 'worst_node.sd')

In [48]:
tmp = dt[dt["Unnamed: 0"] == 125]

In [54]:
dt

,Unnamed: 0,attr_conc.nanmean,attr_conc.nansd,attr_ent.nanmean,attr_ent.nansd,best_node.nanmean,best_node.nansd,class_conc.nanmean,class_conc.nansd,cor.nanmean,...,linear_discr.nanmean.1,linear_discr.nansd.1,naive_bayes.nanmean.1,naive_bayes.nansd.1,one_nn.nanmean.1,one_nn.nansd.1,random_node.nanmean.1,random_node.nansd.1,worst_node.nanmean.1,worst_node.nansd.1
0,48,0.000000,0.000000,0.0,0.000000,0.000000,0.010002,0.0,0.027517,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
1,49,0.000000,0.000000,0.0,0.000000,0.003004,0.000000,0.0,0.000000,0.000000,...,0.0,0.005983,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
2,50,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.007013,0.000000,0.005984,0.000000,0.000000,0.0,0.000000,0.000000,0.0
3,51,0.000000,0.000000,0.0,0.000000,0.002964,0.000000,0.0,0.000000,0.001027,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
4,52,0.000000,0.000000,0.0,0.000000,0.001992,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.001994,0.0
5,53,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.008972,0.0,0.000000,0.000000,0.0
6,54,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.017952,0.0,0.000000,0.000000,0.0
7,55,0.000000,0.000000,0.0,0.000000,0.002992,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
8,56,0.000000,0.000000,0.0,0.000998,0.002994,0.000000,0.0,0.000000,0.000000,...,0.0,0.007950,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
9,57,0.000000,0.000000,0.0,0.000000,0.004082,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.099733,0.000000,0.0,0.000000,0.000000,0.0


In [19]:
import numpy as np

In [83]:
arr_X[:, 1]

array(['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C',
       'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C',
       'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C',
       'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C',
       'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C',
       'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C',
       'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C',
       'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C',
       'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C',
       'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C',
       'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C',
       'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C',
       'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C',
       'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C

In [64]:
C

array([['TN', 'C', 'A', ..., 'Y', '0', '3'],
       ['TN', 'C', 'R', ..., 'Y', '0', '3'],
       ['TN', 'C', 'R', ..., 'Y', '0', '3'],
       ...,
       ['TN', 'C', 'V', ..., 'Y', '0', '3'],
       ['TN', 'C', 'A', ..., 'Y', '0', '3'],
       ['TN', 'C', 'A', ..., 'Y', '500', '3']], dtype=object)

In [53]:
C = C.apply(lambda x: x.astype(object))

In [59]:
C = C.apply(lambda x: x.fillna(x.mode()))

In [61]:
C.apply(lambda x: x.fillna(x.mode()))

,family,product-type,steel,temper_rolling,condition,formability,non-ageing,surface-finish,surface-quality,enamelability,...,corr,blue%2Fbright%2Fvarn%2Fclean,lustre,jurofm,s,p,shape,oil,bore,packing
0,TN,C,A,T,S,2,N,P,G,2,...,NaN,B,Y,NaN,NaN,NaN,COIL,Y,0,3
1,NaN,C,R,NaN,S,2,NaN,NaN,E,NaN,...,NaN,NaN,Y,NaN,NaN,NaN,COIL,NaN,0,NaN
2,NaN,C,R,NaN,S,2,NaN,NaN,E,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,SHEET,NaN,0,NaN
3,NaN,C,A,T,NaN,NaN,NaN,NaN,G,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,COIL,NaN,0,NaN
4,NaN,C,A,T,NaN,NaN,NaN,NaN,G,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,SHEET,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
893,NaN,C,R,NaN,S,3,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,SHEET,NaN,0,NaN
894,NaN,C,R,NaN,S,3,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,SHEET,NaN,0,NaN
895,NaN,C,V,NaN,S,2,NaN,NaN,NaN,2,...,NaN,NaN,NaN,NaN,NaN,NaN,SHEET,NaN,0,NaN
896,NaN,C,A,T,NaN,NaN,NaN,NaN,G,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,COIL,NaN,0,NaN


In [2]:
MFE.valid_metafeatures(groups = ["landmarking"])

('best_node',
 'elite_nn',
 'linear_discr',
 'naive_bayes',
 'one_nn',
 'random_node',
 'worst_node')

In [3]:
# csv_files = glob.glob(f"../data/meta_features/*.{'csv'}")
# df = pd.concat([pd.read_csv(f, index_col = 0) for f in csv_files])


In [209]:
csv_files = glob.glob(f"../data/test/*.{'csv'}")
df = pd.concat([pd.read_csv(f, index_col = 0) for f in csv_files])

In [210]:
df_full = df[df.isna().all(1) == False]

In [71]:
from metafeature_mapping import mapping_general, mapping_stat, mapping_infotheory

mapping = mapping_general | mapping_stat | mapping_infotheory

In [ ]:
df.fillna()

In [139]:
ftrs = list(mapping.keys())

In [140]:
datasets = openml.datasets

In [66]:
from meta_func import BackFiller
import numpy as np
filler = BackFiller()

In [67]:
from pymfe.info_theory import MFEInfoTheory
mfe_info = MFEInfoTheory()
def impute_metafeatures(X, y, df):
    missing_elements = np.unique(np.array(list(zip(*df.loc[:, df.isnull().any()].columns.str.split('.')))))
    for element in missing_elements:
        if element in landmarkers:
            continue
        if element in mapping_infotheory.keys():
            imputing_class = mfe_info
            value = mapping[element]
            func, args = value[0], value[1:]
            C = pd.DataFrame(X).select_dtypes('category')
            C = C.to_numpy()
            if len(args) > 1:
                args = C, y
            else:
                args = C
        else:
            imputing_class = filler
            func = mapping[element]
            args = X
        
        exec_func = getattr(imputing_class, func)
        result = exec_func(args)
        if type(result) == tuple:
            column_str = [f"{element}.mean", f"{element}.sd"]
        else:
            df[element] = result
            continue
        for col, res in zip(column_str, result):
            df[col] = res
    return df

In [193]:
datasets = openml.datasets

In [201]:
df_full = df_full[df_full['attr_conc.mean'].notnull()].copy()
df_full = df_full[df_full['class_conc.mean'].notnull()].copy()

In [72]:
for inx in meta_features.index:

    dataset = datasets.get_dataset(inx, download_all_files=False)
    X, y, _, _ = dataset.get_data(dataset_format="array", target=dataset.default_target_attribute)
    df = pd.DataFrame(columns = meta_features.columns)
    df.loc[inx] = meta_features.loc[inx].values

    row_ = impute_metafeatures(X, y, df)
    meta_features.loc[inx] = row_.values

KeyError: 'best_node'